# LTC3114 Calculations



In [2]:
from __future__ import division
import math


In [4]:
#Inductor Selection 
#Inductor Selection for Buck Mode (worst case Vin = 40V)
Vin = 40
Vout = 5
L = 6.8E-6
f = 1.2E6       
Iout_max = 1   #maximum average output current at 5V

delta_Il = Vout * (1-(Vout/Vin))/(L*f)
print "delta_Il for buck mode = %f A" %delta_Il 

#Il_max is an accepted safety factor for inductor capacity that takes into account heavy load transients.

Il_max = Iout_max + delta_Il/2  
print "The maximum instantaneous current in the inductor will be Il_max = %f A" %Il_max 



delta_Il for buck mode = 0.612745 A
The maximum instantaneous current in the inductor will be Il_max = 1.306373 A


In [13]:
#Here are the numbers from the LTC3114 datasheet for Inductor Selection:

Vin = 40
Vout = 5
L = 6.8E-6
f = 1.2E6
tlow = .05E-6
Ilpp = Vout/L * (Vin-Vout)/Vin * ((1/f)- tlow)
print "Peak to peak ripple current for %f uH inductor in buck mode (according to LTC3114 calcs) is %f A" %(L*1E6, Ilpp)

Peak to peak ripple current for 6.800000 uH inductor in buck mode (according to LTC3114 calcs) is 0.503983 A


In [17]:
#Inductor Selection for Boost Mode (Worst Case Vin = 2.5V which is 3.0V input power and then a 500mV diode drop)  

Vin = 2.5
Vout = 5
L = 6.8E-6
f = 1.2E6  
Iout_max = 1  #maximum output current DC A.  
neff = 0.8

Iboost_dc = Vout*Iout_max/Vin*neff

#Calculate the DC inductor current in boost mode
Ilpp_boost = (Vin/L) * (Vout-Vin)/Vout * ((1/f) - tlow)  

print "The maximum ripple current in boost mode is %f A" %Ilpp_boost  

print "The maximum current through the inductor in boost mode is %f A" %(Iboost_dc + Ilpp_boost/2)






The maximum ripple current in boost mode is 0.143995 A
The maximum current through the inductor in boost mode is 1.671998 A


The inductor chosen for the (3V, 40V) to 5V SMPS must have at least a 1.67A current rating, so round up to 2A current rating.  From the LTC3114 datasheet, the inductor value should be between 4.7uH and 15uH.  Choose 6.8uH.  The SRN5040 family of inductors from Bournes has inductor values including 4.7uH, 6.8uH, 8.2uH that are all rated above 2A.  Choose the SRN5040-6R8M which has 2.3A current rating, 59mOhm DC resistance, 21MHz resonant frequency in a 5mm x 5mm x 4mm package.  This device is in the dock_v2.lbr INDUCTOR_FERRITE_CORE .  

In [21]:
#RUN pin calculations 

#The RUN pin acts as an undervoltage lockout with a 1.2V rising turn on and a 1.1V falling turn off.  
#The RUN pin sinks a max current of 40uA.  Set the current through the divider as 10x 40uA.  

R38 = 1.1/400E-6
R38 = 2.7E3
R4 = 1.9*R38/1.1
print "R38 = %d ohms" %R38
print "R4 = %d ohms" %R4

R38 = 2700 ohms
R4 = 4663 ohms


**JFET LED driver**  

Scrounged up some 2N5457 N Channel JFET transistors.  The zero gate voltage drain current is (1.0mA, 3.0mA, 5.0mA) for Vds = 15V.  Made up the test circuit with a netlist of Vdd, LED_anode; LED_cathode, drain; GND, source.  Three different JFET transistors were tested with a generic TH red LED with the voltage on VCC driven and the current through VCC measured.  

||* Transistor  *||*  Voltage (V) *||*  Current (mA) *||
||      1        ||   2.5          ||   1             ||
||      1        ||   3            ||   2             ||
||      1        ||   4            ||   3             ||
||      1        ||   5            ||   4             || 
||      1        ||   12           ||   4             ||  


The other transistors have similar performance (turn on at about 2.5V, pulls up to 4mA at 5V and then stable above 5V).  So this is a poor man's diode driver.  


In [6]:
#LTC3114 Compensation  
# Following Datasheet reccommendations  
# Boost Mode: Vin = 3V, Vout = 5V, Cout = 20uF, L = 6.8uH, Rload = 5 ohms
Vin = 3
Vout = 5
Rload = 5
Cout = 20E-6
L = 6.8E-6



RHPZ = Vin**2 * Rload/(Vout**2 * 2 * math.pi * 6.8E-6)

print "RHPZ = %f Hz" %RHPZ


RHPZ = 42129.249642 Hz


In [8]:
# Output Load Pole
# Choose loop bandwidth of 1/4 the RHPZ, so fcc = 10kHz
P1 = 1/(2*math.pi*Rload*Cout)
print "P1 = %f Hz" %P1

P1 = 1591.549431 Hz


In [11]:
# Compensation Zero Z1  
Gcs = 1.97
gm = 120E-6
fcc = 10000  # This is the desired closed loop frequency response  

Rz = fcc*Vout*2*math.pi*Cout/(Gcs*gm)
print "Rz = %f ohms" %Rz

Rz = 26578.618051 ohms


In [16]:
# Cp1 capacitor
Cp1 = Rload * Cout / Rz
print "Cp1 = %f nF" %(Cp1*1E9)
Cp1

Cp1 = 3.762423 nF


3.762422854692406e-09

In [20]:
# Now account for the inner compensation loop
# make fcc_prime the new crossover frequency
fcc_prime = (fcc/3)*0.42
print "fcc_prime = %f Hz" %fcc_prime


fcc_prime = 1400.000000 Hz


In [18]:
# run the Rz compensation zero Z1 equation again ...
Rz1 = fcc_prime*Vout*2*math.pi*Cout/(Gcs*gm)
print "Rz1 = %f ohms" %Rz1

Rz1 = 3721.006527 ohms


In [19]:
# new Cp1 value  
Cp1_prime = Rload * Cout / Rz1
print "Cp1_prime = %f nF" %(Cp1_prime * 1E9)


Cp1_prime = 26.874449 nF
